# 機械学習フロー
# 【問題1】クロスバリデーション
事前学習期間は検証データを分割しておき、それに対して指標値を計算することで検証を行っていました。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。


具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。
## データの確認

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

## 【問題3】Kernelからの調査
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。そして、効果があると考えられるものを検証してください。  
全パラメータを用いたランダムフォレストが気に入ったのでそのモデルを使用する。

## 【問題4】高い汎化性能のモデル
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
app_train_notarget = app_train.drop('TARGET', axis=1) # TARGETを落としたもの
target = app_train['TARGET']
test_id = app_test[['SK_ID_CURR']].copy()

In [ ]:
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
# 共通の列にしている（今回は変化なし）

In [ ]:
app_train_notarget.shape, target.shape, app_test.shape

In [ ]:
# どちらがテストデータかわかるようにしている(TARGETを落としたため)
app_train['training_set'] = True 
app_test['training_set'] = False

In [ ]:
app_full = pd.concat([app_train, app_test], axis=0).copy() # 縦方向に結合
app_full = app_full.drop('SK_ID_CURR', axis=1)# IDは特徴量ではないので除外
app_full.shape

くっつけることによってダミーデータ作成や欠損値作成を一括で行うことができる。

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
app_full.dtypes.value_counts()

In [ ]:
# 2値のオブジェクトをダミー変数化
le_count = 0

for col in app_full:
    if app_full[col].dtype == 'object':
         if len(list(app_full[col].unique())) <= 2: # 2個以上でそれやると何が何だかわからなくなるため２個まで
            le.fit(app_full[col])
            app_full[col] = le.transform(app_full[col])
            
            le_count += 1
print('%d columns were labl encoded.' % le_count) 

In [ ]:
app_full = pd.get_dummies(app_full)

In [ ]:
app_full.shape

TARGET と　 FLAG系のデータの相関がないことを確認して削除(略)

In [ ]:
app_train['TARGET'] = target
app_doc_corr = app_train[['TARGET','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4',
                        'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8', 
                        'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 
                        'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16',
                        'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
                        'FLAG_DOCUMENT_21']].copy()

In [ ]:
corr = app_doc_corr.corr()['TARGET'].sort_values()
corr

In [ ]:
corr.abs().sort_values(ascending=False)

相関が高かった  
- FLAG_DOCUMENT_3     0.044346
- FLAG_DOCUMENT_6     0.028602
- FLAG_DOCUMENT_16    0.011615
- FLAG_DOCUMENT_13    0.011583  
以外のものを削除  
なんでわざわざこの特徴量だけ削除しているのかはよくわからなかった。

In [ ]:
app_full = app_full.drop(['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 
                        'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 
                        'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 
                        'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15'], axis=1)

In [ ]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis = 1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0:"Missing Values", 1:"% of Total Values"})
    
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending = False).round(1)
    
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
    
    return mis_val_table_ren_columns

In [ ]:
app_full_miss = missing_values_table(app_full)
app_full_miss.head(20)

欠損値補完の方法として、最頻値補完を採用している。  
70%もないデータがあるにもかかわらず無理矢理特攻していくのは驚きだ  
また、標準化ではなく正規化をしている。

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median') #最頻値で補完
scaler = MinMaxScaler(feature_range=(0, 1)) #最大値1　 最小値0に設定

In [ ]:
for col in app_full.columns:
    app_full[[col]] = imputer.fit_transform(app_full[[col]])
    app_full[[col]] = scaler.fit_transform(app_full[[col]])

In [ ]:
app_full.isnull().sum().sort_values(ascending=False)

In [ ]:
app_train = app_full[app_full['training_set']==1.0]

In [ ]:
app_test = app_full[app_full['training_set']==0.0]

In [ ]:
app_train = app_train.drop('training_set', axis=1)
app_test = app_test.drop('training_set', axis=1)

In [ ]:
app_train.shape

In [ ]:
app_test.shape

In [ ]:
target.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(app_train, target)

In [ ]:
result_rf_model = pd.DataFrame(rf_model.predict_proba(app_test)[:, 1], columns=['TARGET']) # dfにして保存
result_rf = pd.concat([test_id, result_rf_model], axis=1)
RESULT_rf = result_rf.set_index('SK_ID_CURR') # 結合とindexの振り直し
RESULT_rf.to_csv('result rf.csv')

In [ ]:
RESULT_rf

## ランダムサーチ

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# n_estimator = np.arange(50, 300, 50)
# min_samples_split = np.arange(2, 20, 2)
# min_samples_leaf = np.arange(2, 20, 2)
# param = {'n_estimators' : n_estimator, 'min_samples_split' : min_samples_split, 'min_samples_leaf' : min_samples_leaf}
# rf_model_ranserch = RandomForestClassifier(random_state=0, n_jobs=-1)
# ran_rf = RandomizedSearchCV(rf_model_ranserch, param, cv=5, scoring='roc_auc', n_iter=20)
# ran_rf.fit(app_train, target)

In [ ]:
# print('best_score : {}'.format(ran_rf.best_score_)) # このスコアは交差検証の平均値？？？
# print('best_param : {}'.format(ran_rf.best_params_))# 一番良かったのを返す

In [ ]:
# ran_rf = pd.DataFrame.from_dict(ran_rf.cv_results_, orient='index').T # 結果をdataframeにして表示
# ran_rf.sort_values('rank_test_score').head(30)

In [ ]:
# 一番良かった条件で学習
rf_model_fin = RandomForestClassifier(n_estimators=250, min_samples_split=16, min_samples_leaf=18, random_state=0, n_jobs=-1)

In [ ]:
rf_model_fin.fit(app_train.values, target.values)

In [ ]:
rf_model_fin_pred = rf_model_fin.predict_proba(app_test.values)[:, 1]

rf_fin_res = pd.DataFrame(rf_model_fin_pred, columns=['TARGET'])
rf_fin_res_2 = pd.concat([test_id, rf_fin_res], axis=1)
rf_fin_res_3 = rf_fin_res_2.set_index('SK_ID_CURR') # 結合とindexの振り直し
rf_fin_res_3.to_csv('random forest random search.csv')